## Bài tập 1:

In [2294]:
import pandas as pd
import numpy as np

##### Đọc dataset

In [2295]:
data = pd.read_csv('titanic.csv',usecols=['age','fare','survived'])
data.head()

,survived,age,fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


##### Kiểm tra dữ liệu rỗng

In [2296]:
data.isnull().sum()

survived      0
age         177
fare          0
dtype: int64

###### NHẬN XÉT: Biến [AGE] có 177 trường rỗng & [Fare] không có trường rỗng

### 7 kĩ thuật xử lí dữ liệu trống

In [2297]:
# Median
myMedian = data.copy()
median = myMedian['age'].median()
myMedian['age'] = myMedian['age'].fillna(median)

# Mean
myMean = data.copy()
mean = myMean['age'].median()
myMean['age'] = myMean['age'].fillna(mean)

# Mode imputation
myMode = data.copy()
mode = myMode['age'].mode()
myMode['age'] = myMode['age'].fillna(mode)

# Random
myRand = data.copy()
random_samples = myRand['age'].dropna().sample(n=myRand['age'].isnull().sum(),random_state=0)
random_samples.index = myRand[myRand['age'].isnull()].index
myRand.loc[data['age'].isnull(), 'age'] = random_samples

# Đuôi phân bố
myLast = data.copy()
extreme = myLast["age"].mean() + 3 * myLast["age"].std()
myLast["age"] = myLast["age"].fillna(extreme)

# Tạo giá trị bất kì
myAnyVal = data.copy()
myAnyVal["age"] = myAnyVal["age"].fillna(24)

# Tạo đặc trưng mới
myNewVar = data.copy()
myNewVar['age_NAN'] = np.where(myNewVar['age'].isnull(), 1, 0)
myNewVar["age"].fillna(myNewVar["age"].median(), inplace=True)

In [2298]:
before = [myMedian, myMedian, myMode, myRand, myLast, myAnyVal, myNewVar]

### Xử lí ngoại lệ

In [2299]:
data_inline = data.copy() ## clone data de so sanh
# age (phân bố chuẩn)
upper_age = data['age'].mean() + 3 * data['age'].std()
lower_age = data['age'].mean() - 3 * data['age'].std()
print([lower_age, upper_age])



[-13.880374349943303, 73.27860964406094]


In [2300]:
#fare (phân bố lệch)
IQR = data["fare"].quantile(0.75) - data["fare"].quantile(0.25)
lower_fare = data['fare'].quantile(0.25) - 3 * IQR
upper_fare = data['fare'].quantile(0.75) + 3 * IQR
print([lower_fare, upper_fare])

[-61.358399999999996, 100.2688]


In [2301]:
#### Loại bỏ các giá trị outline
data_inline.loc[data_inline['age'] >= 73, 'age'] = 73
data_inline.loc[data_inline['fare'] >= 100, 'fare'] = 100


In [2302]:
# Median
myMedian = data_inline.copy()
median = myMedian['age'].median()
myMedian['age'] = myMedian['age'].fillna(median)

# Mean
myMean = data_inline.copy()
mean = myMean['age'].median()
myMean['age'] = myMean['age'].fillna(mean)

# Mode imputation
myMode = data_inline.copy()
mode = myMode['age'].mode()
myMode['age'] = myMode['age'].fillna(mode)

# Random
myRand = data_inline.copy()
random_samples = myRand['age'].dropna().sample(n=myRand['age'].isnull().sum(),random_state=0)
random_samples.index = myRand[myRand['age'].isnull()].index
myRand.loc[data['age'].isnull(), 'age'] = random_samples

# Đuôi phân bố
myLast = data_inline.copy()
extreme = myLast["age"].mean() + 3 * myLast["age"].std()
myLast["age"] = myLast["age"].fillna(extreme)

# Tạo giá trị bất kì
myAnyVal = data_inline.copy()
myAnyVal["age"] = myAnyVal["age"].fillna(24)

# Tạo đặc trưng mới
myNewVar = data_inline.copy()
myNewVar['age_NAN'] = np.where(myNewVar['age'].isnull(), 1, 0)
myNewVar["age"].fillna(myNewVar["age"].mean(), inplace=True)

In [2303]:
after = [myMedian, myMean, myMode, myRand, myLast, myAnyVal, myNewVar]

### Logistic Regression

In [2304]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
def Logistic_Reg(data):
    acc_scores = []
    for i in range(10):
        X_train, X_test, Y_train, Y_test = train_test_split(
            data[['age', 'fare']].fillna(0),
            data['survived'],test_size=0.3, 
            random_state=i)
        classifier = LogisticRegression()
        classifier.fit(X_train, Y_train)
        Y_pred = classifier.predict(X_test)
        acc_scores.append(accuracy_score(Y_test, Y_pred))
    return round(np.mean(acc_scores) * 100, 2)

### Chuẩn hoá dữ liệu

#### Chuẩn hoá z-score 


In [2314]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
def zscore_norm(data1):
    data2 = data1[['age', 'fare', 'survived']].copy()
    scaler = StandardScaler()
    data2 = pd.DataFrame(scaler.fit_transform(data2), columns=data2.columns)
    data2['survived'] = data1['survived']
    return data2

str(Logistic_Reg(zscore_norm(myNewVar))) + '%'

'66.64%'

In [2306]:
col = ['median', 'mean', 'mode', 'random', 'last', 'any_val', 'new_variable']
for i in range(7):
    before[i] = Logistic_Reg(before[i])
    after[i] = Logistic_Reg(after[i])

### Bảng thống kê độ độ chính xác của từng cặp ( kĩ thuật - trường hợp ) 
###### ( Đơn vị % )

In [2307]:
x = pd.DataFrame([before, after],index=['outliers', 'inliners'], columns=col)
x

,median,mean,mode,random,last,any_val,new_variable
outliers,65.34,65.34,65.56,65.45,66.38,65.34,65.34
inliners,66.68,66.68,67.24,66.75,66.72,66.60,66.72


#### Nhận xét :
- Các phương pháp Median, Mean, Mode, NewVariable đưa ra kết quả thấp (65.34% DCX). Do 3 phương pháp Median, Mean, Mode và NewVariable(thay dữ liệu cột trống bằng Mean) làm thay đổi phương sai của dữ liệu. 
- Đã dùng phương pháp chuẩn hoá tuy nhiên độ chính xác chỉ đạt 66.64% ( thấp hơn chưa chuẩn hoá ) với kĩ thuật tạo đặc trưng mới => chưa chính xác ( chưa tìm ra nguyên nhân )